# 程式說明
作業環境: `jupyter notebook`

使用語言:`Python3`

請安裝`pip install opencv-python`   目前使用版本1.16.2

# 操作說明
---
透過OpenCv取得影像，並傳送到微軟Azure，篩選出車牌內容

# 影像存取說明
#### 導入函式庫
import cv2  # 導入函式庫
#### 打開視窗
cv2.namedWindow('視窗名稱',選項與設定)
#### 關閉視窗
cv2.destroyWindow('視窗名稱')
#### 讀取影像
img = cv2.imread('圖檔.jpg',選項與設定)
#### 顯示影像
cv2.imshow('視窗名稱',img)
#### 加入 等待時間
cv2.waitKey(1000) #1000毫秒 = 1分鐘
#### 儲存影像
cv2.imwrite(存檔路徑,影像變數[,選項])

# 將上述的功能結合起來應用

In [15]:
import cv2

try:
    img = cv2.imread('2.jpg')             # 讀取圖片
    img_small = cv2.resize(img, (300, 100)) # 改變尺寸
    cv2.imshow('Frame1', img)               # 顯示原圖
    cv2.imshow('Frame2', img_small)         # 顯示新圖
    cv2.waitKey(1000)                          # 等待
    cv2.destroyAllWindows()                 # 關閉視窗
    try:
        cv2.imwrite('small.jpg', img_small) # 儲存影像
        print('saved')
    except:
        print('Error：write')
except:
    print('Error：read')

saved


---
# 攝影機操作說明
#### 攝影機
攝影機物件 = cv2.VideoCapture(0)#若有多個攝影機可以決定幾號，N為數字
capture = cv2.VideoCapture(0)
#### 確認攝影機有沒有打開回傳一個布林值
capture .isOpened()
#### 從攝影機讀取影像，回傳一個布林值、影像變數
success, img = capture .read()
#### 從影像變數儲存影像
cv2.imshow('視窗',影像變數)
cv2.imwrite(存檔路徑,影像變數[,選項])
#### 關閉攝影機
capture.release()

In [1]:
import cv2                      # 匯入 cv2 套件

capture = cv2.VideoCapture(0)   # 建立攝影機物件
if capture.isOpened():          
    while True:
        sucess, img = capture.read()        # 讀取影像
        if sucess:
            cv2.imshow('Frame', img)        # 顯示影像
        k = cv2.waitKey(100)                # 等待按鍵輸入
        if k == ord('s') or k == ord('S'):  # 按下 S(s)
            cv2.imwrite('shot.jpg', img)    # 儲存影像
            print('稍後在此加入車牌辨識功能')   
            
        if k == ord('q') or k == ord('Q'):  # 按下 Q(q) 結束迴圈
            print('exit')
            cv2.destroyAllWindows()         # 關閉視窗
            capture.release()               # 關閉攝影機
            break
else:
    print('開啟攝影機失敗')

 

稍後在此加入車牌辨識功能
稍後在此加入車牌辨識功能
稍後在此加入車牌辨識功能
exit


# 使用Azure進行辨識
---
    Azure取得資訊:
    端點

    https://westcentralus.api.cognitive.microsoft.com/vision/v1.0

    https://westcentralus.api.cognitive.microsoft.com/vision/v2.0

    https://westcentralus.api.cognitive.microsoft.com/vision/v2.1

    金鑰 1: b70b1bbfd92e41dabc36a1aa723164e7

    金鑰 2: 6705b734640241b68f53caf1186ccd19

    辨識respone1 = requests.post(請求路徑,請求標題,請求主題)
    辨識格式 可以分為兩種
    Handwritten  = 辨識手寫字
    Printed = 辨識正體字(印刷)

In [ ]:
# 車牌辨識範例
base = https://westcentralus.api.cognitive.microsoft.com/vision/v2.0
recog_url  = f'{base}/recognizeText?mode=Printed'
response1 = requests.post(recog_url,請求標頭,請求主體)

In [ ]:
#請求主體的型態內容範例
key = 'b70b1bbfd92e41dabc36a1aa723164e7'
headers_stream = { 'Ocp-Apim-Subscription-key':key
                                        'Content-Type': 'application/octet-stream'} #body內容類型

response1 = requests.post(請求路徑,
                                                        headers = headers_stream,#指定標頭
                                                        請求主題)

In [ ]:
#請求主題範例(我們的影像資料)
#傳送一張已經轉為bytes物件的影像資料
response1 = requests.post(recong_url.
                                                        headers = headers_stream,
                                                        data = img_bytes)
result_url = response1.headers['Operation-Lococation']

In [ ]:
#取得辨識結果
response2 = requests.get(請求路徑,請求標頭)
#請求路徑 = 'Operation-Lococation'
#請求標頭 = '伺服器的key資訊'

In [5]:
#時間模組(sleep)-讓程式進入休眠
import time
#time.sleep(n) 時間秒數
#使用範例
import time 
for i in range(0,3):
    print('Sleep...')
    time.sleep(i)
    
print('起床')

Sleep...
Sleep...
Sleep...
起床


# 辨識圖片中車牌內容
---
1. 使用OpenCV獲取本地圖片
2. 使用Azure 辨識出裡面的文字
3. 辨識出圖片中可能包含的非數字型態的文字

In [17]:
import requests
import cv2
import time
import re

base = 'https://westcentralus.api.cognitive.microsoft.com/vision/v2.0'
recog_url = f'{base}/recognizeText?mode=Printed'
key = 'b70b1bbfd92e41dabc36a1aa723164e7'
headers = {'Ocp-Apim-Subscription-Key': key}                            # 查看結果的請求標頭
headers_stream = {'Ocp-Apim-Subscription-Key': key,                     # 辨識的請求標頭
                  'Content-Type': 'application/octet-stream'}  
def get_license(img):
    img_encode = cv2.imencode('.jpg', img)[1]       # 將 img 編碼為 jpg 格式，[1]返回資料, [0]返回是否成功
    img_bytes     = img_encode.tobytes()                # 再將資料轉為 bytes, 此即為要傳送的資料
    r1 = requests.post(recog_url,            # 發出 POST 
                              headers = headers_stream, 
                              data        = img_bytes)
    if r1.status_code != 202:                # 202 代表接受請求
        print(r1.json())
        return '請求失敗'
    #辨識請求成功
    result_url = r1.headers['Operation-Location'] # 取得查看結果的請求路徑
    r2 = requests.get(result_url, headers = headers) # 發出 GET 請求
    while r2.status_code == 200 and r2.json()['status'] != 'Succeeded':
        r2 = requests.get(result_url, headers = headers)    # 繼續發出 GET
        time.sleep(0.5)
        print('status: ', r2.json()['status'])      # 顯示辨識狀態
    #辨識完成#
    carcard = ''  # 紀錄車牌
    lines = r2.json()['recognitionResult']['lines']
    for i in range(len(lines)):
        text = lines[i]['text'] # 取得辨識文字
        m = re.match(r'^[\w]{2,4}[-. ][\w]{2,4}$', text)    # 匹配是否為車牌格式
        if m != None:   # 匹配成功
            carcard = m.group()
            return carcard
    if carcard == '':   # 無匹配結果
        return '找不到車牌'

## 程式說明
--- 
- 13. 自訂函式接受到img 後，先使用cv2的`imencode()`方法將影像進行JPEG的格式編碼並暫存於記憶中，回傳結果的索引值
- 14. 編碼後的的影像資料再經由`tobytes()`方法轉為bytes物件，即可透過post傳送
- 24~27. 當r2 回應物件的`status`內容並非`Succeeded`時,等待0.5秒再發送一次請求直到`status`為`Succeeded`為止，離開迴圈執行下列事件
- 30~32. 若辨識成功，則由r2回應物件中取出鍵為'lines'的串列，再以for迴圈取出其中所有text的內容
- 33. 將所有text 匹配車牌的常規表達式，內容必須以2-4個英數字為開頭，中間可以式'-'、'.'、'**空白**'符號，最後再以2-4個英文字結尾

In [18]:
try:
    img = cv2.imread('car2.jpg') # 讀取圖片
    print('status:  Start')
    text = get_license(img)     # 辨識圖片中的車牌
    print('車牌：', text)
    cv2.imshow('Frame', img)    # 顯示圖片
    cv2.waitKey(0)              # 等待
    cv2.destroyAllWindows()     # 關閉視窗
except:
    print('讀取圖片失敗')

status:  Start
車牌： RAT.2182


## 攝影機進行拍照再進行車牌辨識
---
操作 :  
Ctrl + Enter = 執行程式(開啟攝影機)

S  = 代表儲存影像

Q = 關閉攝影機

In [22]:
import requests
import cv2
import time
import re

base = 'https://westcentralus.api.cognitive.microsoft.com/vision/v2.0'
recog_url = f'{base}/recognizeText?mode=Printed'
key = 'b70b1bbfd92e41dabc36a1aa723164e7'
headers = {'Ocp-Apim-Subscription-Key': key}                            # 查看結果的請求標頭
headers_stream = {'Ocp-Apim-Subscription-Key': key,                     # 辨識的請求標頭
                  'Content-Type': 'application/octet-stream'}  
def get_license(img):
    img_encode = cv2.imencode('.jpg', img)[1]       # 將 img 編碼為 jpg 格式，[1]返回資料, [0]返回是否成功
    img_bytes     = img_encode.tobytes()                # 再將資料轉為 bytes, 此即為要傳送的資料
    r1 = requests.post(recog_url,            # 發出 POST 
                              headers = headers_stream, 
                              data        = img_bytes)
    if r1.status_code != 202:                # 202 代表接受請求
        print(r1.json())
        return '請求失敗'
    #辨識請求成功
    result_url = r1.headers['Operation-Location'] # 取得查看結果的請求路徑
    r2 = requests.get(result_url, headers = headers) # 發出 GET 請求
    while r2.status_code == 200 and r2.json()['status'] != 'Succeeded':
        r2 = requests.get(result_url, headers = headers)    # 繼續發出 GET
        time.sleep(0.5)
        print('status: ', r2.json()['status'])      # 顯示辨識狀態
    #辨識完成#
    carcard = ''  # 紀錄車牌
    lines = r2.json()['recognitionResult']['lines']
    for i in range(len(lines)):
        text = lines[i]['text'] # 取得辨識文字
        m = re.match(r'^[\w]{2,4}[-. ][\w]{2,4}$', text)    # 匹配是否為車牌格式
        if m != None:   # 匹配成功
            carcard = m.group()
            return carcard
    if carcard == '':   # 無匹配結果
        return '找不到車牌'

capture = cv2.VideoCapture(0)#建立攝影機物件
if capture.isOpened():
    while True:
        sucess, img = capture.read()
        if sucess :
            cv2.imshow('Frame', img)
        k = cv2.waitKey(100)
        if k == ord('s') or k ==ord ('S'):
            cv2.imwrite('shot.jpg', img)
            text = get_license(img)
            print('車牌 :',text)
            
        if k == ord('q') or k == ord('Q'):
            print('exit')
            cv2.destroyAllWindows()
            capture.release()
            break
            
else:
    print('開啟攝影機失效')

車牌 : 9C.9508
exit
